Implementación distribuida y paralelizada de un algoritmo de Machine Learning

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

import numpy as np
from pyspark import RDD

import time
import json


def readFile(filename):
    def transform_to_floats(line:str) -> np.ndarray:
        # Importante. Retornar como tupla
        element = [float(x) for x in line.split(',')]
        return (np.array(element))
    
    data_raw = contex_global.textFile(filename)
    return data_raw.map(transform_to_floats)


def _calculate_mean(rdd:RDD, total_rows:int) -> np.ndarray:
    def reduce_function(value1, value2):
        return value1[:11] + value2[:11]
    
    return rdd.reduce(reduce_function) / total_rows

def _calculate_stdev(rdd:RDD, mean:np.ndarray, total_rows:int) -> np.ndarray:
    def map_function(element):
        return (element[:11] - mean)**2

    def reduce_function(value1, value2):
        return value1[:11] + value2[:11]
    
    rdd_aux = rdd.map(map_function)
    return (rdd_aux.reduce(reduce_function) / total_rows)**(1/2)


def normalize(RDD_Xy:RDD) -> RDD:
    def map_function(element, mean, desvest):
        x_values = (element[:11] - mean) / desvest
        return np.concatenate((x_values, [element[-1]]))

    m = RDD_Xy.count()
    mean = _calculate_mean(RDD_Xy, m)
    desvest = _calculate_stdev(RDD_Xy, mean, m)
    return RDD_Xy.map(lambda element: map_function(element, mean, desvest))


def _sigmoid(z):
    return 1 / (1 + np.exp(-z))

def _map_label_y_hat_diffs(element, W:np.ndarray, b:np.ndarray):
    """Añade después de la y(etiqueta) el y_hat(ŷ) y la diferencia ŷ-y que se
    utiliza para calcular tanto el coste como las derivadas de W y b. Es necesario
    mantener el orden en el RDD.
    """
    dot_product = 0
    n = W.size
    for i in range(n):
        dot_product += element[i] * W[i]
    
    y_hat = float(_sigmoid(dot_product + b))
    diff = y_hat - element[-1]
    to_append = [y_hat, diff]

    return (np.append(element, to_append)) 

def _calculate_y_hat_and_diffs(RDD_Xy, W:np.ndarray, b:np.ndarray) -> RDD:
    return RDD_Xy.map(lambda element: _map_label_y_hat_diffs(element, W, b))

def _calculate_cost_J(RDD_XY) -> float:
    def _map_cost_function_J(element):
        label, y_hat = element
        #DEBUG. Cambiar log10 a log en la versión final
        return label * np.log(y_hat) + (1 - label) * np.log(1 - y_hat)
    
    # Cuidado con los índices, que es aquí donde se hace la "vista".
    RDD_label_y_hat = RDD_XY.map(lambda element: (element[-3], element[-2]))
    aux_costs = RDD_label_y_hat.map(_map_cost_function_J)
    m = RDD_label_y_hat.count()
    return -aux_costs.reduce(lambda x, y: x + y) / m

def _map_label_y_pred(element, W:np.ndarray, b:np.ndarray):
    dot_product = 0
    n = W.size
    for i in range(n):
        dot_product += element[i] * W[i]
    # ¿Hace falta mantener el orden etiqueta <-> y_hat para calcular
    # accuracy? Sí, para ver cuántas veces acierta.
    y_hat = float(_sigmoid(dot_product + b))
    THRESHOLD = 0.5
    if y_hat > THRESHOLD:
        y_pred = 1
    else:
        y_pred = 0
    return (element[-1], y_pred)

def predict(W:np.ndarray, b:np.ndarray, RDD_Xy) -> RDD:
    return RDD_Xy.map(lambda element: _map_label_y_pred(element, W, b))

def accuracy(W, b, RDD_Xy:RDD) -> float:
    def map_function(element):
        label, y_pred = element
        if label == y_pred:
            return 1
        else:
            return 0
    predictions_rdd = predict(W, b, RDD_Xy)
    aux_acc = predictions_rdd.map(map_function)
    m = aux_acc.count()
    accuracy = aux_acc.reduce(lambda x,y: x + y) / m
    return accuracy

def train(RDD_Xy:RDD, iterations, learning_rate):
    def b_map(element):
        return element[-1] # Se corresponde con las diferencias
    
    def W_map(element):
        return np.array([element[-1] * element[i] for i in range(num_W)])
    
    m = RDD_Xy.count()
    MODEL_PARAMS = 12 # 11 pesos + 1 sesgo
    num_W = MODEL_PARAMS - 1

    W = 2 * np.random.rand(num_W) - 1
    b = 2 * np.random.rand(1) - 1

    for it in range(iterations):
        rdd_xy_y_hat_diffs = _calculate_y_hat_and_diffs(RDD_Xy, W, b)

        dW_aux = rdd_xy_y_hat_diffs.map(W_map)
        dW = dW_aux.reduce(lambda x,y: x + y) / m
        W = W - learning_rate * dW
        
        diffs_label_hat = rdd_xy_y_hat_diffs.map(b_map)
        db = diffs_label_hat.reduce(lambda x,y: x + y) / m
        b = b - learning_rate * db

        cost = _calculate_cost_J(rdd_xy_y_hat_diffs)
        cost_list.append(cost)
        print(f'[train] Iteración {it}, Coste = {cost}')

    return W,b

import json
import numpy as np

def save_data(resultado, filename):
    # Convertir todos los tipos `numpy` a tipos de Python en el diccionario `resultado`
    for key, value in resultado.items():
        if isinstance(value, (np.integer, np.floating)):
            resultado[key] = value.item()  # Convertir a `int` o `float` de Python
        elif isinstance(value, np.ndarray):
            resultado[key] = value.tolist()  # Convertir arrays de `numpy` a listas

    try:
        with open(filename, 'r') as infile:
            data = json.load(infile)
    except FileNotFoundError:
        data = []  # Si el archivo no existe, comenzamos con una lista vacía
    
    # Añadir el resultado convertido a la lista `data`
    data.append(resultado)

    with open(filename, 'w') as outfile:
        json.dump(data, outfile, indent=4)



#FILE_NAME = '../../0-SPAI/1-datos/botnet_tot_syn_l.csv'
#FILE_NAME = 'botnet_sample.csv'
FILE_NAME = '../../0-SPAI/1-datos/botnet_100k.csv'

N_ITER = 20

OUTPUT_FILE = 'learning_rate_100k.json'


for learning_rate in np.arange(1 ,10, 1):
    if 'session_global' in globals():
        session_global.stop()
    
    session_global = SparkSession.builder.master(f'local[*]').getOrCreate()
    contex_global = session_global.sparkContext

    start = time.time()
    print('Learning Rate ', learning_rate)

    # read data
    data_raw_rdd = readFile(FILE_NAME)

    # standarize
    normal_data = normalize(data_raw_rdd)

    # train
    cost_list = []
    W, b = train(normal_data, N_ITER, learning_rate)

    acc = accuracy(W, b, normal_data)
    print('Accuracy ', acc)

    end = time.time()
    runtime = end - start
    print('Tiempo de ejecución (s) ', runtime)

    results = {
        "learn_rate": learning_rate,
        "accuracy": acc,
        "runtime": runtime,
        "costs": cost_list
    }

    save_data(results, OUTPUT_FILE)


24/10/26 19:30:42 WARN Utils: Your hostname, daniel-fpga resolves to a loopback address: 127.0.1.1; using 192.168.0.201 instead (on interface enp2s0f2)
24/10/26 19:30:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/26 19:30:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/spark/python/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


Learning Rate  1


[train] Iteración 0, Coste = 0.9418597471572787


[train] Iteración 1, Coste = 0.5636649914572222


[train] Iteración 2, Coste = 0.40473652030380136


...

[train] Iteración 16, Coste = 0.16108105805834774


[train] Iteración 17, Coste = 0.16104648198001364


[train] Iteración 18, Coste = 0.16102319924368222


[train] Iteración 19, Coste = 0.16100731775227786


Accuracy  0.93821
Tiempo de ejecución (s)  256.2562675476074
